<a href="https://colab.research.google.com/github/ratmcu/wiki_ner/blob/master/info_box.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pyahocorasick
!pip install fuzzyset
from ahocorasick import Automaton
import fuzzyset
import urllib
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
import nltk
import re
import os
import pandas as pd
nltk.download('punkt')

     |████████████████████████████████| 317kB 4.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
     |████████████████████████████████| 378kB 5.0MB/s 
     |████████████████████████████████| 51kB 22.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/d8/36/9a/8f1cac047c7c3b03dce3d5434ed0088bfd8da8aeca615dfb4c
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
  Stored in directory: /root/.cache/pip/wheels/a7/4e/77/da46da4c5aece69b701cf45e6e4fe56ff98bfbdcc7b271d03b
Successfully built fuzzyset python-levenshtein texttable
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
class PageContents():
    def __init__(self, url):
        self.quote_page = url
        try:
            self.page = urllib.request.urlopen(self.quote_page)
            self.soup = BeautifulSoup(self.page, 'html.parser')
        except:
            self.table = False
        try:
            self.side_pane = self.soup.find('table', attrs={'class': 'infobox vcard'})
            self.table_entry_list = self.side_pane.find('tbody').find_all('tr')
            self.table = True
        except:
            self.table = False
    #       print(self.table_entry_list)
    #       self.table_entry_list = self.soup.find('tbody').find_all('tr')
    
    def get_party(self):
#       side_pane = self.soup.find('table', attrs={'class': 'infobox vcard'})
#       table_entry_list = side_pane.find_all('tr')
        party = ''
        for table_entry in self.table_entry_list:
            for child in table_entry.children:
                if child.text == 'Political party':
                    party = child.next_sibling.contents[0].get('title')
#                   print(party)

In [0]:
class InfoCard():
    def __init__(self, page_content):
        self.info_table = {}
        if not page_content.table:
            raise RuntimeError('page content is incomplete') from error
        for table_entry in page_content.table_entry_list:
            try:
                row_left = table_entry.find('th', attrs={'scope': 'row'})
                row_right = row_left.next_sibling
                self.info_table[row_left.text] = self.get_text_parts(self.get_kids(row_right))
            except:
                pass
        self.info_table = self.filter_info_scrapes(self.info_table)
        print('info card is scraped successfully')
        
    def get_kids(self, html_mother):
        kid_list = []
        try:
            kids = html_mother.children
            for kid in kids:
                kid_list.append(self.get_kids(kid))
        except:
            return html_mother
        return kid_list 

    def get_text_parts(self, text_lists):
        text_parts = []
        if type(text_lists) == list:
            for element in text_lists:
                text_parts.extend(self.get_text_parts(element))
        else:
            text_parts.append(text_lists)
        return(text_parts)

    def filter_info_scrapes(self, scape_dict):
        mask_dict = {}
        for key, val in scape_dict.items():
            mask_dict[key] = []
            for i, element in enumerate(scape_dict[key]):
                fnd = [c in element for c in u'()[]\n\xa0']
                if not True in fnd:
                    mask_dict[key].append(element)  
        return mask_dict    

In [0]:
!pip install spacy
!python -m spacy download en_core_web_sm
import spacy
import regex


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_sm

    You can now load the model via spacy.load('en_core_web_sm')



In [0]:
# can we use snorkle to get this entity scraping done??? 
# if so it'd be better to have the lable functions in a ready way to generally working on all the pages!!
# 
class PrivateEntities():
    def __init__(self, info_card):
        self.info_dict = info_card.info_table
        self._get_entity_dict()
        self._extract_entities()
        
    def _extract_entities(self):
        # TREAT everyone the same? or call a seperate function per each entity?
        # leave the not found entities as empty lists so we can use the html scraper to fill them
        # lets treat everything generally, ultimate filtering happens at the comparison stage(if theres one)
        for entity_key in self.entity_dict.keys():
            self._pick_entity(self.entity_dict[entity_key], entity_key)
            print(self.entity_dict[entity_key])
    
    def _pick_entity(self, entity_list, entity_key):
        nlp = spacy.load("en_core_web_sm")
#         print(entity_list)
        # Find named entities, phrases and concepts
        entity_list.append([])
#         print(entity_list[0][0])
        
        if entity_key == 'BIRTH_PLACE':
            mask_str = ''
            for scrape in self.info_dict[entity_list[0][0]]:
                doc = nlp(str(scrape))
                if doc.ents and doc.ents[0].label_ == 'DATE':
                    mask_str = mask_str + 'd'
                elif str(scrape) == ', ':
                    mask_str = mask_str + 's'
                elif self._is_name(str(scrape)):
                    mask_str = mask_str + 't'
                else:
                    mask_str = mask_str + 'u'
            print(mask_str)
            indices = self._get_target_indices([r'[s][t]',r'[d][t]'], mask_str)  
            for index in indices:
                entity_list[1].append(self.info_dict[entity_list[0][0]][index])
        else:    
            for scrape in self.info_dict[entity_list[0][0]]:
                doc = nlp(str(scrape))
    #             print(scrape)
                if doc.ents and doc.ents[0].label_ == entity_list[0][1]:
                    entity_list[1].append(doc.ents[0].text)
    #                 print(doc.ents[0].text)
                elif not self._entity_noise(entity_key, str(scrape)):
                    entity_list[1].append(str(scrape))
                else:
                    pass
                
    def _get_target_indices(self, pattern_list, mask_str):
        '''TODO: if the target is inside the pattern'''
        indices = []
        for pattern in pattern_list:
            m_iter = re.finditer(pattern, mask_str)
            if m_iter:
                for m in m_iter: 
                    indices.append(m.start()+1)
        return indices        
        
    def _entity_noise(self, entity_key, text):
        try:
            return getattr(self, '_filter_{0}'.format(entity_key))(text)
        except:
            print('filter function for {0} not found'.format(entity_key))
            return True
    
    def _get_entity_dict(self):
        self.entity_dict = {}
        if 'Born' in self.info_dict.keys():
            self.entity_dict['NAME'] = [['Born', 'PERSON']]
            self.entity_dict['BIRTH_DATE'] = [['Born', 'DATE']]
            self.entity_dict['BIRTH_PLACE'] = [['Born', 'GPE']]
        if 'Children' in self.info_dict.keys():
            self.entity_dict['CHILDREN'] = [['Children', 'PERSON']]
        if 'Spouse(s)' in self.info_dict.keys():
            self.entity_dict['SPOUSES'] = [['Spouse(s)', 'PERSON']]
        if 'Parents' in self.info_dict.keys():
            self.entity_dict['PARENTS'] = [['Parents', 'PERSON']]
        if 'Education' in self.info_dict.keys():
            self.entity_dict['EDUCATION'] = [['Education', 'ORG']]
    
    def _filter_NAME(self, text):
        '''cannot simply filter noise due to many possible candidates'''
        return True
    def _filter_BIRTH_DATE(self, text):
        return True
    def _filter_BIRTH_PLACE(self, text):
        return True
    def _filter_CHILDREN(self, text):
#         return False # let's assume we don't find any noisy text under info boxes children till we do an analysis
        return not self._is_name(text)
    def _filter_SPOUSES(self, text):
        return not self._is_name(text)
    def _filter_PARENTS(self, text):
        return not self._is_name(text)
    def _filter_EDUCATION(self, text):
        return True
    def _is_name(self, text):
        p = regex.compile(r"\p{Lu}") # To support (currently) 1702 uppercase letters
#         p = regex.compile(r"[[:upper:]]") # To support (currently) 1822 uppercase letters
        if p.match(text):
            return True
        else:
            return False
        
from unittest import TestCase

class PrivateEntitiesTest(TestCase):
    def setUp(self):
        self.pvt_ent = PrivateEntities()


In [0]:
class example():
    def __init__(self):
        print('initialized')
        getattr(self, 'chase_{0}'.format('clout'))('flacky')
    def chase_clout(self, chaser):
        print('chasing the clout with {0}'.format(chaser))

ex = example()

initialized
chasing the clout with flacky


In [0]:
# pg = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
# pg = PageContents('https://en.wikipedia.org/wiki/Barack_Obama')
pg = PageContents('https://en.wikipedia.org/wiki/Joachim_Gauck')


info_card = InfoCard(pg)
# print(type(str(info_card.info_table['Born'][0])))
pe = PrivateEntities(info_card)
# pe.entity_dict

info card is scraped successfully
[['Born', 'PERSON'], []]
[['Born', 'DATE'], ['1940-01-24', '24 January 1940']]
ddtstst
[['Born', 'GPE'], ['Mecklenburg', 'Nazi Germany', 'Rostock']]
[['Children', 'PERSON'], []]
[['Spouse(s)', 'PERSON'], ['Gerhild Radtke  ']]


In [0]:
info_card.info_table

{'Born': ['1940-01-24',
  '24 January 1940',
  'Rostock',
  ', ',
  'Mecklenburg',
  ', ',
  'Nazi Germany'],
 'Chancellor': ['Angela Merkel'],
 'Children': ['4'],
 'Constituency': ['Alliance 90 List'],
 'Domestic partner': ['Daniela Schadt', ' '],
 'Other politicalaffiliations': ['New Forum', '/', 'Alliance 90', ' '],
 'Political party': ['Independent', ' '],
 'Preceded by': ['Position established'],
 'Signature': [],
 'Spouse(s)': ['Gerhild Radtke  '],
 'Succeeded by': ['Marianne Birthler']}

In [0]:
entity_dict = {}
entity_dict['NAME'] = [['Born', 'PERSON']]
entity_dict['BIRTH_DATE'] = [['Born', 'DATE']]
entity_dict['BIRTH_PLACE'] = [['Born', 'GPE']]
entity_dict['CHILDREN'] = [['Children', 'PERSON']]
entity_dict['SPOUSES'] = [['Spouse(s)', 'PERSON']]
entity_dict['PARENTS'] = [['Born', 'PERSON']]
entity_dict['EDUCATION'] = [['Education', 'ORG']]
entity_dict.keys()

dict_keys(['NAME', 'BIRTH_DATE', 'BIRTH_PLACE', 'CHILDREN', 'SPOUSES', 'PARENTS', 'EDUCATION'])

In [0]:
pg = PageContents('https://en.wikipedia.org/wiki/Barack_Obama')
# pg = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
info_card = InfoCard(pg)
info_card.info_table

info card is scraped successfully


{'Awards': ['Nobel Peace Prize', '2009', 'Profile in Courage Award'],
 'Born': ['Barack Hussein Obama II',
  '1961-08-04',
  'August 4, 1961',
  'Honolulu',
  ', ',
  'Hawaii',
  ', U.S.'],
 'Children': ['Malia', 'Sasha'],
 'Education': ['Occidental College',
  'Columbia University',
  'BA',
  'Harvard University',
  'JD'],
 'Parents': ['Barack Obama Sr.', 'Ann Dunham'],
 'Political party': ['Democratic'],
 'Preceded by': ['Alice Palmer'],
 'Relatives': ['Obama family'],
 'Residence': [],
 'Signature': [],
 'Spouse(s)': ['Michelle Robinson', 'm.', '1992'],
 'Succeeded by': ['Kwame Raoul'],
 'Vice President': ['Joe Biden'],
 'Website': ['Official website', 'Obama Foundation', 'White House Archives']}

In [0]:
def get_kids(html_mother):
    kid_list = []
    try:
        kids = html_mother.children
        for kid in kids:
            kid_list.append(get_kids(kid))
    except:
        return html_mother
    return kid_list

def get_text_parts(text_lists):
    text_parts = []
    if type(text_lists) == list:
        for element in text_lists:
            text_parts.extend(get_text_parts(element))
    else:
        text_parts.append(text_lists)
    return(text_parts)

def filter_info_scrapes(scape_dict):
    mask_dict = {}
    for key, val in scape_dict.items():
#         print(key)
#         print(scape_dict[key])
        mask_dict[key] = []
        for i, element in enumerate(scape_dict[key]):
#             print(element)
#             mask_dict[key].append(element)
            fnd = [c in element for c in u'()[]\n\xa0']
            if not True in fnd:
#                 print('!!!')
# #                 mask_dict[key][-1] = '1'
# #                 del val[i]
#             else:
                mask_dict[key].append(element)  
    return mask_dict

True

In [0]:
pg = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
# pg = PageContents('https://en.wikipedia.org/wiki/Ashraf_Ghani')
# pg = PageContents('https://en.wikipedia.org/wiki/Oracle_Corporation')


In [0]:
info_table = {}
start = True

for table_entry in pg.table_entry_list:
#     for child in table_entry.children:
    if table_entry.text == 'Personal details':
        start = True
    if start:    
#             print(child.text)
        try:
            row_left = table_entry.find('th', attrs={'scope': 'row'})
            row_right = row_left.next_sibling
#             print(row_left.text)
#             print(row_right)
#             print(get_text_parts(get_kids(row_right)))
            info_table[row_left.text] = get_text_parts(get_kids(row_right))
#             break
        except:
            pass

# info_table
# mthr
info_table = filter_info_scrapes(info_table)
# print(info_table)
# info_table['Children'][0]
info_table


{'Awards': ['List of honors and awards'],
 'Born': ['Donald John Trump',
  '1946-06-14',
  'June 14, 1946',
  'Queens',
  ', ',
  'New York City'],
 'Children': ['Donald Jr.', 'Ivanka', 'Eric', 'Tiffany', 'Barron'],
 'Education': ['The Wharton School', 'BS', ' in ', 'Econ.'],
 'Net worth': [],
 'Occupation': ['Politician',
  'businessman',
  'real estate developer',
  'television personality'],
 'Other politicalaffiliations': ['Democratic', 'Reform', 'Independent'],
 'Parents': ['Fred Trump', 'Mary Anne MacLeod'],
 'Political party': ['Republican'],
 'Preceded by': ['Barack Obama'],
 'Relatives': ['Trump family'],
 'Residence': ['White House', 'Trump Tower', 'Full list'],
 'Signature': [],
 'Spouse(s)': ['Ivana Zelníčková',
  'm.',
  '1977',
  '; ',
  'div.',
  '1992',
  'Marla Maples',
  'm.',
  '1993',
  '; ',
  'div.',
  '1999',
  'Melania Knauss',
  'm.',
  '2005'],
 'Vice President': ['Mike Pence'],
 'Website': ['Official website',
  'White House website',
  'Presidential Twitter'

In [0]:
filter_info_scrapes(info_table)
info_table

{'Awards': ['List of honors and awards'],
 'Born': ['Donald John Trump',
  ' (',
  '1946-06-14',
  ') ',
  'June 14, 1946',
  ' (age\xa072)',
  'Queens',
  ', ',
  'New York City'],
 'Children': ['\n',
  'Donald Jr.',
  '\n',
  'Ivanka',
  '\n',
  'Eric',
  '\n',
  'Tiffany',
  '\n',
  'Barron',
  '\n'],
 'Education': ['The Wharton School', ' (', 'BS', ' in ', 'Econ.', ')'],
 'Net worth': ['US$3.1\xa0billion (March 2019)', '[a]'],
 'Occupation': ['Politician',
  'businessman',
  'real estate developer',
  'television personality'],
 'Other politicalaffiliations': ['\n',
  'Democratic',
  ' (until 1987, 2001–2009)',
  '\n',
  'Reform',
  ' (1999–2001)',
  '\n',
  'Independent',
  ' (2011–2012)',
  '\n'],
 'Parents': ['Fred Trump', 'Mary Anne MacLeod'],
 'Political party': ['Republican', ' (1987–1999, 2009–2011, 2012–present)'],
 'Preceded by': ['Barack Obama'],
 'Relatives': ['Trump family'],
 'Residence': ['\n',
  'White House',
  ' (official)',
  '\n',
  'Trump Tower',
  ' (personal)'

In [0]:
[c in info_table['Spouse(s)'][-1] for c in u'()[],;.\n']

[False, False, False, False, False, False, False, True]